In [36]:
#import tarfile
#import os
#os.chdir('/home/vagrant/data/mine')
#print os.getcwd()

#print os.listdir(os.getcwd())
#fh = tarfile.open('review_polarity.tar.gz', 'r:gz')

#fh.extractall()


/home/vagrant/data/mine
['review_polarity.tar.gz']


In [2]:
negdata=sc.wholeTextFiles('/home/vagrant/data/mine/txt_sentoken/neg/').map(lambda a:(1 if 'pos' in a[0] else 0,a[1]))
posdata=sc.wholeTextFiles('/home/vagrant/data/mine/txt_sentoken/pos/').map(lambda a:(1 if 'pos' in a[0] else 0,a[1]))
#posdata=sc.union([sc.textFile('/home/vagrant/data/mine/txt_sentoken/pos/' + "\n" + f) for f in os.listdir('/home/vagrant/data/mine/txt_sentoken/pos/')])
#posdata=sc.wholeTextFiles('/home/vagrant/data/mine/txt_sentoken/pos/'+os.listdir('/home/vagrant/data/mine/txt_sentoken/pos/')[0])
data=negdata.union(posdata)
data.count()

2000

In [3]:

data=data.cache()
negdata=negdata.cache()
posdata=posdata.cache()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import load_files
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [3]:

from pyspark.mllib.feature import HashingTF
documents=data.map(lambda a:a[1].split(' '))

hashingTF = HashingTF()
tf = hashingTF.transform(documents)

In [4]:
documents.take(1)

[[u'the',
  u'first',
  u'image',
  u'in',
  u'"',
  u'final',
  u'fantasy',
  u':',
  u'the',
  u'spirits',
  u'within',
  u'"',
  u'is',
  u'a',
  u'computer-animated',
  u'close-up',
  u'of',
  u'a',
  u'human',
  u'eye',
  u'.',
  u"\nit's",
  u'a',
  u'beautiful',
  u'piece',
  u'of',
  u'work',
  u',',
  u'remarkably',
  u'detailed',
  u'and',
  u'quite',
  u'convincing',
  u'.',
  u'\nwhen',
  u'the',
  u'picture',
  u'pulls',
  u'back',
  u'to',
  u'reveal',
  u'the',
  u'owner',
  u'of',
  u'the',
  u'eye',
  u',',
  u'however',
  u',',
  u'things',
  u'change',
  u'.',
  u'\nthe',
  u'young',
  u'woman',
  u'has',
  u'mesmerizing',
  u'hair',
  u',',
  u'although',
  u'it',
  u'hangs',
  u'too',
  u'artfully',
  u'?',
  u'even',
  u'by',
  u'movie',
  u'standards',
  u'?',
  u'to',
  u'be',
  u'believed',
  u'.',
  u'\nthe',
  u'facial',
  u'features',
  u'are',
  u'more',
  u'detailed',
  u'than',
  u'any',
  u'computer-animation',
  u'seen',
  u'to',
  u'date',
  u',',
  u'

In [5]:
tf.first()

SparseVector(1048576, {19: 1.0, 2424: 1.0, 3015: 1.0, 3932: 12.0, 4415: 7.0, 6292: 1.0, 9314: 1.0, 13954: 1.0, 15590: 1.0, 18549: 1.0, 24165: 5.0, 24208: 1.0, 28000: 1.0, 28017: 2.0, 36751: 2.0, 36754: 2.0, 36757: 2.0, 38019: 1.0, 38034: 1.0, 43230: 1.0, 48642: 1.0, 50570: 14.0, 50573: 6.0, 50583: 12.0, 50591: 1.0, 52134: 1.0, 52237: 1.0, 52431: 1.0, 57166: 5.0, 57787: 1.0, 64361: 1.0, 66096: 1.0, 68798: 1.0, 69414: 4.0, 69913: 1.0, 74605: 2.0, 75990: 1.0, 76149: 2.0, 80932: 2.0, 81775: 3.0, 84450: 1.0, 85077: 1.0, 90808: 1.0, 91755: 1.0, 93759: 1.0, 94976: 1.0, 97255: 1.0, 104251: 4.0, 104426: 1.0, 110080: 1.0, 110522: 4.0, 113471: 1.0, 115245: 46.0, 117402: 1.0, 117409: 1.0, 123489: 1.0, 124562: 1.0, 124666: 1.0, 126987: 1.0, 129375: 1.0, 132052: 1.0, 134398: 1.0, 136113: 1.0, 141912: 4.0, 146975: 1.0, 149770: 1.0, 156287: 1.0, 158643: 1.0, 159034: 1.0, 160065: 1.0, 160915: 1.0, 167617: 1.0, 168184: 2.0, 171836: 1.0, 178362: 2.0, 180063: 1.0, 183576: 1.0, 190145: 1.0, 191617: 1.0, 19

In [4]:
from pyspark.mllib.feature import IDF

tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [7]:
#labeledTfidfPoints.map(lambda a:a.features).take(5)

In [5]:
#make labeledpoints
from pyspark.mllib.regression import LabeledPoint
#?LabeledPoint
labeledTfidfPoints=tfidf.zipWithIndex().map(lambda (a,b):(b,a)).join(data.map(lambda a:a[0]).zipWithIndex().map(lambda (a,b):(b,a))).map(lambda (a,b):LabeledPoint(b[1],b[0]))

In [6]:
#labeledpoint function
def giveLabeledPoints(keyValuePair):
    return LabeledPoint(keyValuePair[0],keyValuePair[1][0])

In [7]:
#create labeledpoints by using map on the rdd
#labeledTfidfPoints=labeledTfidf.map(lambda a:giveLabeledPoints(a))
#labeledTfidfPoints=labeledTfidf.map(lambda a:LabeledPoint(a[0],a[1][0]))

In [8]:
#create training, validation and testing data
weights = [.8, .1, .1]
seed = 42
labeledTfidfPoints.cache()
rawTrainData, rawValidationData, rawTestData = labeledTfidfPoints.randomSplit(weights,seed)

In [9]:
rawTrainData.cache()
rawValidationData.cache()
rawTestData.cache()


PythonRDD[28] at RDD at PythonRDD.scala:43

In [10]:
from pyspark.mllib.classification import SVMWithSGD

svm = SVMWithSGD.train(data=rawTrainData, iterations=10, step=1.0, regParam=0.01, miniBatchFraction=1.0, initialWeights=None, regType='l2', intercept=False)

In [11]:
svmLabelsAndPredsTrain = rawTrainData.map(lambda p: (p.label, svm.predict(p.features)))
#Overall True Prediction Rate
svmLabelsAndPredsTrain.filter(lambda (a,b):a==b).count()/float(rawTrainData.count())

0.9956140350877193

In [12]:
#True Negatives
print svmLabelsAndPredsTrain.filter(lambda (a,b):a==0).filter(lambda (a,b):a==b).count()/float(rawTrainData.filter(lambda a:a.label==0).count())
#True Positives
print svmLabelsAndPredsTrain.filter(lambda (a,b):a==1).filter(lambda (a,b):a==b).count()/float(rawTrainData.filter(lambda a:a.label==1).count())

0.996216897856
0.99501867995


In [13]:
a=sc.parallelize(tfidf.zipWithIndex().map(lambda (a,b):(b,a)).take(3))
b=sc.parallelize(data.map(lambda a:a[0]).zipWithIndex().map(lambda (a,b):(b,a)).take(3))

In [17]:
a.collect()

[(0,
  SparseVector(1048576, {19: 3.7513, 2424: 2.6316, 3015: 3.2194, 3932: 0.3839, 4415: 2.1873, 6292: 2.1898, 9314: 2.4029, 13954: 4.5569, 15590: 4.5569, 18549: 6.2151, 24165: 1.6415, 24208: 3.0581, 28000: 1.8648, 28017: 2.6123, 36751: 0.2442, 36754: 0.0973, 36757: 0.3426, 38019: 1.533, 38034: 0.7428, 43230: 2.4253, 48642: 6.5028, 50570: 0.035, 50573: 0.2761, 50583: 0.0421, 50591: 0.5478, 52134: 5.6555, 52237: 3.8172, 52431: 2.8919, 57166: 0.4741, 57787: 4.4233, 64361: 2.2308, 66096: 4.1049, 68798: 5.992, 69414: 2.4006, 69913: 3.5071, 74605: 1.9335, 75990: 5.2035, 76149: 1.021, 80932: 3.962, 81775: 20.7248, 84450: 4.8934, 85077: 2.0759, 90808: 6.9083, 91755: 3.7728, 93759: 4.5104, 94976: 5.1165, 97255: 6.5028, 104251: 1.9013, 104426: 2.571, 110080: 5.992, 110522: 1.5535, 113471: 5.0365, 115245: 0.069, 117402: 1.9489, 117409: 2.3544, 123489: 5.2988, 124562: 4.2692, 124666: 3.4905, 126987: 2.2687, 129375: 2.2833, 132052: 3.1706, 134398: 1.4009, 136113: 0.6987, 141912: 0.7861, 146975: 4

In [18]:
#Make Predictions on Validation set
svmLabelsAndPredsVal=rawValidationData.map(lambda p: (p.label, svm.predict(p.features)))

In [19]:
#True Negatives
print svmLabelsAndPredsVal.filter(lambda (a,b):a==0).filter(lambda (a,b):a==b).count()/float(rawValidationData.filter(lambda a:a.label==0).count())
#True Positives
print svmLabelsAndPredsVal.filter(lambda (a,b):a==1).filter(lambda (a,b):a==b).count()/float(rawValidationData.filter(lambda a:a.label==1).count())

0.724770642202
0.927272727273


In [4]:
sqlContext=SQLContext(sc)
from pyspark.mllib.linalg import Vectors

dataDf=sqlContext.createDataFrame(data.zipWithIndex().map(lambda (a,b):(b,a[1],float(a[0]))),['id','text','label'])
#trainDf,valDf,testDf = dataDf.randomSplit(weights,seed)

In [21]:
dataDf.first()

Row(id=0, text=u'the first image in " final fantasy : the spirits within " is a computer-animated close-up of a human eye . \nit\'s a beautiful piece of work , remarkably detailed and quite convincing . \nwhen the picture pulls back to reveal the owner of the eye , however , things change . \nthe young woman has mesmerizing hair , although it hangs too artfully ? even by movie standards ? to be believed . \nthe facial features are more detailed than any computer-animation seen to date , but the result is more reminiscent of a very well-crafted doll than anything human . \nshe is pretty , but bland , and not nearly expressive enough to come off like a person . \nall the characters in " final fantasy " are like that . \nof the core group , the younger white men and women are all athletic , attractive and indistinct , like applicants for a tv reality show . \nthe black man is taller and burlier , and the aging scholar is bald , with wrinkles and a beard . \nnone of them appear to be based

In [5]:
weights=[.75,.25]
seed=45
trainDf,valDf = dataDf.randomSplit(weights,seed)

In [36]:
?Tokenizer

In [6]:
################################################
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.feature import NGram
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator



# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
ngram = NGram( inputCol="words", outputCol="nGrams")
hashingTF = HashingTF(inputCol=ngram.getOutputCol(), outputCol="features")
idf = IDF(minDocFreq=3, inputCol="features", outputCol="idf")
nb = LogisticRegression()
paramGrid = ParamGridBuilder().addGrid( ngram.setN,[1, 2,3]).build()

pipeline = Pipeline(stages=[tokenizer, ngram,hashingTF, idf, nb])



#model=pipeline.fit(trainDf)


cv = CrossValidator(estimator=pipeline, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=MulticlassClassificationEvaluator(), 
                    numFolds=4)

cvModel = cv.fit(trainDf)

/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/pyspark/ml/classification.py:207: UserWarning: weights is deprecated. Use coefficients instead.
  warnings.warn("weights is deprecated. Use coefficients instead.")


In [7]:
#asd=testDf.take(1)[]
#m=model
#prediction=model.transform(valDf)
result = cvModel.transform(valDf)

In [ ]:
#prediction_df = result.select("text", "label", "prediction")

In [8]:
selection=result.select("id", "text", "prediction","label")

In [18]:
selection.first()

<bound method CrossValidatorModel.explainParams of CrossValidatorModel_44bdb91268dbe36e8d2a>

In [10]:
selection.map(lambda a:(a[2],a[3])).filter(lambda (a,b):a==b).count()/float(selection.count())

0.7603305785123967

In [35]:
svmLabelsAndPredsTrain.filter(lambda (a,b):a==b).count()/float(rawValidationData.count())

7.255707762557078

In [34]:
import re
def rem(x):
    return re.sub('[^A-Za-z0-9|-]+', ' ',x)
    #eturn re.sub(r'[^\\W\S]','',x)
print rem('asf dsf  ds gdsg fg 7-s t&*%^*&%db')

asf dsf ds gdsg fg 7-s t db


In [33]:
from pyspark.ml.feature import NGram
from pyspark.ml.feature import StopWordsRemover
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.mllib.classification import LabeledPoint
from pyspark.ml.feature import  Tokenizer
#from pyspark.ml.feature import IDF
#from pyspark.ml import Pipeline
#from pyspark.ml.classification import LogisticRegression
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#from pyspark.ml.tuning import ParamGridBuilder
#from pyspark.ml.tuning import CrossValidator
ngram = NGram(inputCol="words", outputCol="ngrams",n=2)
tok=Tokenizer(inputCol='text',outputCol='word')
stopRem=StopWordsRemover(inputCol='word',outputCol='words')
#w=data
w=sqlContext.createDataFrame(dataDf.map(lambda a:(a[0],rem(a[1]),a[2])),['id','text','label'])
w=tok.transform(w)
w=stopRem.transform(w)
w=ngram.transform(w)
hashingTF = HashingTF()
hashedtf=hashingTF.transform(w.map(lambda a:a[5]))
hashedtf.cache()
hashedtfidf = IDF().fit(hashedtf)
hashedtfidf = hashedtfidf.transform(hashedtf)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 118.0 failed 1 times, most recent failure: Lost task 0.0 in stage 118.0 (TID 224, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/pyspark/rdd.py", line 1293, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-33-8061568180ae>", lin

In [10]:
from pyspark.mllib.classification import LabeledPoint
LPtfidf=w.map(lambda a:(a[0],a[2])).join(hashedtfidf.zipWithIndex().map(lambda (a,b):(b,a))).map(lambda(a,b):LabeledPoint(b[0],b[1]))
LPtfidf.cache()

PythonRDD[40] at RDD at PythonRDD.scala:43

In [11]:
weights=[.75,.25]
seed=45
trainDf,valDf = dataDf.randomSplit(weights,seed)

In [12]:
trainDfLP,valDfLP = LPtfidf.randomSplit(weights,seed)
#w.map(lambda a:(a[0],a[2])).join(hashedtfidf.zipWithIndex().map(lambda (a,b):(b,a))).map(lambda(a,b):LabeledPoint(b[0],b[1])).take(1)#.map(lambda (a,b):LabeledPoint(b))

In [18]:
from pyspark.mllib.classification import SVMWithSGD
svm = SVMWithSGD.train(data=trainDfLP, iterations=100, step=1.0, regParam=1000, miniBatchFraction=1.0, initialWeights=None, regType='l2', intercept=False)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server

In [19]:
pred=valDfLP.map(lambda p: (p.label, svm.predict(p.features)))
pred.cache()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server

In [20]:
#pred.take(3)
#pred.map(lambda a:a[0]).distinct().collect()
print pred.filter(lambda (a,b):a==b).count()/float(valDfLP.count())
#sc.parallelize([1,0,1]).zipWithIndex().map(lambda (a,b):(b,a)).collect()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/bin/spark-1.6.1-bin-hadoop2.6/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server

In [174]:
from pyspark.mllib.feature import HashingTF
documents=data.map(lambda a:a[1].split(' '))

hashingTF = HashingTF()
tf = hashingTF.transform(documents)
from pyspark.mllib.feature import IDF

# ... continue from the previous example
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [386]:
print LPtfidf.take(1)
aa=sc.parallelize([(1,'a')])
bb=sc.parallelize([(1,'c')])
aa.leftOuterJoin(bb).collect()

[LabeledPoint(0.0, (1048576,[120,1105,2756,5642,10435,10779,14043,15356,19960,25180,26055,27250,28184,32488,34371,34528,44152,44471,45671,46240,47388,48344,50031,50900,51828,52029,55285,57093,60932,61639,62127,64061,65685,66260,66659,68211,69925,70604,72299,72477,73312,75282,79595,80736,82747,84977,85967,87333,88315,96629,96888,103111,104838,109891,110454,110873,112703,122692,124971,128463,129612,130857,131534,135430,137315,138594,140910,144834,149925,150819,151476,157210,165668,167917,169325,170137,173670,174195,176861,176972,188514,190554,191201,197789,200082,202893,202998,203517,207697,208189,208293,208380,209042,211482,212521,212955,214400,218830,227772,228485,233481,233878,236687,237801,243505,243552,243937,244410,246561,247092,247512,247518,248481,248674,255248,256531,257780,261947,266666,266735,269559,270334,274991,275547,279736,279934,282946,283128,291695,291822,292053,293913,294554,295731,295979,298369,299448,304346,308489,309196,314615,315206,319775,321990,321994,327355,33152

[(1, ('a', 'c'))]

In [302]:
tok=Tokenizer(inputCol='text',outputCol='words')
w=tok.transform(dataDf)
from pyspark.ml.feature import NGram
ngram = NGram(inputCol="words", outputCol="ngrams",n=2)
w=ngram.transform(w)
s=rawTrainData.take(1)[0]
#s
dataDf.take(1)

[Row(id=0, text=u'the first image in " final fantasy : the spirits within " is a computer-animated close-up of a human eye . \nit\'s a beautiful piece of work , remarkably detailed and quite convincing . \nwhen the picture pulls back to reveal the owner of the eye , however , things change . \nthe young woman has mesmerizing hair , although it hangs too artfully ? even by movie standards ? to be believed . \nthe facial features are more detailed than any computer-animation seen to date , but the result is more reminiscent of a very well-crafted doll than anything human . \nshe is pretty , but bland , and not nearly expressive enough to come off like a person . \nall the characters in " final fantasy " are like that . \nof the core group , the younger white men and women are all athletic , attractive and indistinct , like applicants for a tv reality show . \nthe black man is taller and burlier , and the aging scholar is bald , with wrinkles and a beard . \nnone of them appear to be base

In [201]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.cross_validation import train_test_split
from sklearn import metrics

# The training data folder must be passed as first argument
#languages_data_folder = sys.argv[1]
dataset = load_files('/home/vagrant/data/mine/txt_sentoken',encoding='utf8',decode_error='replace')


In [202]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import load_files
from sklearn.cross_validation import train_test_split
from sklearn import metrics
#from sklearn.


docs_train, docs_test, y_train, y_test = train_test_split(
        dataset.data, dataset.target, test_size=0.25, random_state=None)

pipeline=Pipeline([('vect',TfidfVectorizer(min_df=3,max_df=.95)),('clf',LinearSVC(C=1000))])

In [ ]:
    # TASK: Build a grid search to find out whether unigrams or bigrams are
    # more useful.
    # Fit the pipeline on the training set using grid search for the parameters
#?GridSearchCV
parameters = {
        'vect__ngram_range': [(1, 1), (1, 2)],
    }
#print gridSearch.get_params()
#?GridSearchCV
gridSearch=GridSearchCV(pipeline,parameters,n_jobs=1)
gridSearch.fit(docs_train,y_train)


In [217]:
# TASK: print the cross-validated scores for the each parameters set
# explored by the grid search
print(grid_search.grid_scores_)

# TASK: Predict the outcome on the testing set and store it in a variable
# named y_predicted
y_predicted = grid_search.predict(docs_test)

# Print the classification report
print(metrics.classification_report(y_test, y_predicted,
                                        target_names=dataset.target_names))

# Print and plot the confusion matrix
cm = metrics.confusion_matrix(y_test, y_predicted)
print(cm)

[mean: 0.81533, std: 0.00817, params: {'vect__ngram_range': (1, 1)}, mean: 0.83067, std: 0.01496, params: {'vect__ngram_range': (1, 2)}]
             precision    recall  f1-score   support

        neg       0.97      0.99      0.98       251
        pos       0.99      0.97      0.98       249

avg / total       0.98      0.98      0.98       500

[[248   3]
 [  7 242]]


In [211]:
import pickle
f=open('/home/vagrant/mine/grid','r')
grid_search=pickle.load(f)

In [230]:
################################################
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.feature import NGram
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator



# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
ngram = NGram( inputCol="words", outputCol="nGrams")
hashingTF = HashingTF(inputCol=ngram.getOutputCol(), outputCol="features")
idf = IDF(minDocFreq=3, inputCol="features", outputCol="idf")
nb = LogisticRegression()
paramGrid = ParamGridBuilder().addGrid( ngram.setN,[1, 2,3]).build()

pipeline = Pipeline(stages=[tokenizer, ngram,hashingTF, idf, nb])



#model=pipeline.fit(trainDf)


cv = CrossValidator(estimator=pipeline, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=MulticlassClassificationEvaluator(), 
                    numFolds=4)

cvModel = cv.fit(trainDf)

[Row(id=0, text=u'the first image in " final fantasy : the spirits within " is a computer-animated close-up of a human eye . \nit\'s a beautiful piece of work , remarkably detailed and quite convincing . \nwhen the picture pulls back to reveal the owner of the eye , however , things change . \nthe young woman has mesmerizing hair , although it hangs too artfully ? even by movie standards ? to be believed . \nthe facial features are more detailed than any computer-animation seen to date , but the result is more reminiscent of a very well-crafted doll than anything human . \nshe is pretty , but bland , and not nearly expressive enough to come off like a person . \nall the characters in " final fantasy " are like that . \nof the core group , the younger white men and women are all athletic , attractive and indistinct , like applicants for a tv reality show . \nthe black man is taller and burlier , and the aging scholar is bald , with wrinkles and a beard . \nnone of them appear to be base